# Naiwny klasyfikator bayesowski

ZADANIE:

Wyobraźmy sobie test na chorobę X. Testy wykrywa prawdidłowo 90% chorych, ale jeśli testowi poddaje się osoba zdrowa myli się w 30% przypadków. Choroba X dotyka średnio 10% populacji. 

Jakie jest prawdodpodobieństwo, że osoba, która poddała się testowi i otrzymała wynik pozytywny (chora), jest zdrowa?

P(A|B) = (P(B|A) * P(A)) / P(B)

## Budowa modelu na danych rzeczywistych

Po pierwsze, musimy zaimportować dane. Pobierzmy sobie zatem zbiory z githuba.

In [ ]:
!git clone https://github.com/matzim95/ML-datasets


fatal: destination path 'ML-datasets' already exists and is not an empty directory.


In [5]:
import pandas as pd

def load_dataset(filename, class_column, index_col=None):
    dataset = pd.read_csv(f'ML-datasets/{filename}.csv', index_col=index_col)
    dataset['class'] = dataset[class_column].astype('category').cat.codes
    classes = dataset.pop(class_column).unique()
    return dataset, classes

### Wczytanie zbioru

Glass

Wine

### Normalizacja / standaryzacja

Możemy już przejść do zdefiniowania modelu i jego walidacji

Ten sposób porównywania nie jest zbyt czytelny, dlatego stosujemy metryki oraz macierze błędów. Przygotujmy funkcję:

A na co się przydaje ta dyskretyzacja? Sprawdźmy!

Zdefiniujmy sobie różne funkcje do dyskretyzacji, korzystając z pandasowych:
* pd.cut()
* pd.qcut()

Funkcje przeprowadzające kategoryzację:

## Granice decyzyjne na sztucznie wygenerowanym zbiorze: